# Units in MDMC

## SYSTEM Units

All quantities passed as arguments and returned as outputs in MDMC have units associated with them. In general, these units are consistent across all functions and are referred to as the `SYSTEM` units associated with a physical property:

In [1]:
from MDMC.common import units

print(units.SYSTEM)

{'LENGTH': 'Ang', 'TIME': 'fs', 'MASS': 'amu', 'CHARGE': 'e', 'ANGLE': 'deg', 'TEMPERATURE': 'K', 'ENERGY': 'kJ / mol', 'FORCE': 'kJ / Ang mol', 'PRESSURE': 'Pa', 'ENERGY_TRANSFER': 'meV', 'ARBITRARY': 'arb'}


For example, both the `Atom` and `Universe` expect units of `Ang` for `LENGTH` properties (the `position` and `dimensions` arguments respectively):

In [2]:
from MDMC.MD import Atom, Universe

print(help(Atom))
print(help(Universe))

Supported DL_POLY version 4.10
Help on class Atom in module MDMC.MD.structures:

class Atom(Structure)
 |  Atom(element: 'str', position: "Union['list[float]', 'tuple[float]', np.ndarray]" = (0.0, 0.0, 0.0), velocity: "Union['list[float]', 'tuple[float]', np.ndarray]" = (0.0, 0.0, 0.0), charge: 'float' = None, **settings: 'dict')
 |  
 |  A single atom
 |  
 |  Parameters
 |  ----------
 |  element : str
 |      The atomic element label.
 |  position : list, tuple, numpy.ndarray, optional
 |      A 3 element `list`, `tuple` or ``array`` with the position in units of
 |      ``Ang``. The default is ``(0., 0., 0.)``.
 |  velocity : list, tuple, numpy.ndarray, optional
 |      A 3 element `list`, `tuple` or ``array`` with the velocity in units of
 |      ``Ang / fs``. Note that if set, the velocity of atoms in the MD engine will
 |      be scaled when creating a `Simulation` in order to ensure the
 |      temperature is accurate. Otherwise, if the velocities of all `Atom`
 |      objects 

Similarly, attributes returned also have the units documented. In this case, the unit for `velocity` is a compound unit for the properties `LENGTH` and `TIME` (`Ang` and `fs`):

In [3]:
print(Atom('H').velocity)

[0. 0. 0.] Ang / fs


## Exceptional Cases

There are some cases where the units expected by an MDMC object are not `SYSTEM` due to external conventions. For example, the `SYSTEM` unit for `ANGLE` is `deg`, but `HarmonicPotential` expects units of `kJ mol^-1 rad^-2` when used for a `BondAngle`. As before, this information is available from the help function:



In [4]:
from MDMC.MD import HarmonicPotential

print(help(HarmonicPotential))

Help on class HarmonicPotential in module MDMC.MD.interaction_functions:

class HarmonicPotential(InteractionFunction)
 |  HarmonicPotential(equilibrium_state: float, potential_strength: float, **settings: dict)
 |  
 |  Harmonic potential for bond stretching, and angular and improper dihedral
 |  vibration, with the form:
 |  
 |  .. math::
 |  
 |      E = K(r-r_0)^2
 |  
 |  As ``HarmonicPotential`` can be used with several different ``Interaction``
 |  types, the ``Interaction`` type must be specified, so that the correct units
 |  can be assigned to the ``equilibrium_state`` and ``potential_strength``
 |  parameters.
 |  
 |  Parameters
 |  ----------
 |  equilibrium_state : float
 |      The equilibrium state of the object in either ``Ang`` or ``degrees``,
 |      depending on the ``interaction_type`` passed.
 |  potential_strength : float
 |      The potential strength in units of ``kJ mol^-1 Ang^-2`` (linear) or
 |      ``kJ mol^-1 rad^-2`` (angular/improper), depending on the


Another example is `energy_resolution` being expected in units of `ueV` rather than the `meV` used for `ENERGY_TRANSFER` elsewhere:

In [5]:
from MDMC.control import Control

print(help(Control))

Help on class Control in module MDMC.control.control:

class Control(builtins.object)
 |  Control(simulation: MDMC.MD.simulation.Simulation, exp_datasets: List[dict], fit_parameters: MDMC.MD.parameters.Parameters, minimizer_type: str = 'MMC', FoM_options: dict = None, reset_config: bool = True, MD_steps: int = None, equilibration_steps: int = 0, verbose: int = 0, print_all_settings: bool = False, **settings: dict)
 |  
 |  Controls the MDMC refinement
 |  
 |  Parameters
 |  ----------
 |  simulation : Simulation
 |      Performs a simulation for a given set of potential ``Parameter``
 |      objects.
 |  exp_datasets : list of dicts
 |      Each `dict` represents an experimental dataset, containing the
 |      following keys:
 |        - ``file_name`` (`str`) the file name
 |        - ``type`` (`str`) the type of observable
 |        - ``reader`` (`str`) the reader required for the file
 |        - ``weighting`` (`float`) the weighting of the dataset to be used in
 |          the Figu

## Converting Units

The conversion factor required to get to MDMC `SYSTEM` units from a general unit can be accessed by creating a new `Unit`:

In [6]:
general_unit = units.Unit('kcal / Ang mol')
SYSTEM_unit = units.SYSTEM[general_unit.physical_property]
print('To convert from `{0}` to `{1}`, multiply by a factor of {2}'
      ''.format(general_unit, SYSTEM_unit, general_unit.conversion_factor))
print('To convert from `{1}` to `{0}`, divide by a factor of {2}'
      ''.format(general_unit, SYSTEM_unit, general_unit.conversion_factor))

To convert from `kcal / Ang mol` to `kJ / Ang mol`, multiply by a factor of 4.184
To convert from `kJ / Ang mol` to `kcal / Ang mol`, divide by a factor of 4.184


For example, to create an `Atom` with a position in `nm` rather `Ang`, then access that value in `m`:

In [7]:
import numpy as np

position_nm = np.array([1., 1., 1.,])
position_ang = position_nm * units.Unit('nm').conversion_factor
atom = Atom('H', position=position_ang)
print('Atom position in `m` is {}'.format(np.array(atom.position) / units.Unit('m').conversion_factor))

Atom position in `m` is [1.e-09 1.e-09 1.e-09]
